# Khởi động thư viện Spark

In [1]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('data_processing').getOrCreate()

# Đọc dữ liệu

In [2]:
data=spark.read.csv('C:\Customer-churn.csv',inferSchema=True,header=True)

# Xem thông tin bảng dữ liệu

In [3]:
from pyspark.sql.functions import *
#xem dữ liệu dòng và cột
print(data.count(), len(data.columns))

7043 21


In [4]:
#thông tin lược đồ dữ liệu
data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [5]:
#xem số cột trong dữ liệu
data.columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

In [6]:
#hiển thị 5 dòng đầu tiên của dữ liệu
data.show(5)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|          No|No phone service|            DSL|            No|         Yes|              No|         No|    

In [7]:
#khai thác thông tin dữ liệu (các miêu tả)
data.describe().show()

+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043|  7043|              7043|   7043|      7043|     

# Thống kê các giá trị trung bình, giá trị nhỏ nhất, lớn nhất

In [8]:
# nhóm dữ liệu theo tiêu chí công cụ tìm kiếm, nước
data.groupBy('InternetService').count().show()

data.groupBy('PaymentMethod').count().show()

+---------------+-----+
|InternetService|count|
+---------------+-----+
|    Fiber optic| 3096|
|             No| 1526|
|            DSL| 2421|
+---------------+-----+

+--------------------+-----+
|       PaymentMethod|count|
+--------------------+-----+
|Credit card (auto...| 1522|
|        Mailed check| 1612|
|Bank transfer (au...| 1544|
|    Electronic check| 2365|
+--------------------+-----+



In [9]:
# nhóm nước và tìm giá trị trung bình
data.groupBy('InternetService').mean().show()

+---------------+-------------------+-----------------+-------------------+
|InternetService| avg(SeniorCitizen)|      avg(tenure)|avg(MonthlyCharges)|
+---------------+-------------------+-----------------+-------------------+
|    Fiber optic|0.26841085271317827|32.91795865633075|  91.50012919896615|
|             No|0.03407601572739188|30.54718217562254| 21.079193971166454|
|            DSL|0.10698058653448989|32.82156133828996|  58.10216852540261|
+---------------+-------------------+-----------------+-------------------+



## sử dụng thư viện StringIndexer

In [10]:
#Chuyển dữ liệu từ text sang số
from pyspark.ml.feature import StringIndexer
Platform_indexer = StringIndexer(inputCol='InternetService', outputCol = 'InternetService_Num').fit(data)
data = Platform_indexer.transform(data)

In [11]:
#xem dữ liệu
data.show(3, False)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+-------------------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines   |InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod   |MonthlyCharges|TotalCharges|Churn|InternetService_Num|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+-------------------+
|7590-VHVEG|Female|0            |Yes    |No        |1     |No          |No phone service|DSL            |No           

In [12]:
#Chuyển dữ liệu từ text sang số
from pyspark.ml.feature import StringIndexer
Platform_indexer = StringIndexer(inputCol='PaymentMethod', outputCol = 'PaymentMethod_Num').fit(data)
data = Platform_indexer.transform(data)

In [13]:
#xem dữ liệu
data.show(3, False)

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+-------------------+-----------------+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines   |InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract      |PaperlessBilling|PaymentMethod   |MonthlyCharges|TotalCharges|Churn|InternetService_Num|PaymentMethod_Num|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+-------------------+-----------------+
|7590-VHVEG|Female|0            |Yes    |No        |1     |No   

In [14]:
#Chuyển dữ liệu từ text sang số
from pyspark.ml.feature import StringIndexer
Platform_indexer = StringIndexer(inputCol='gender', outputCol = 'gender_Num').fit(data)
data = Platform_indexer.transform(data)

In [15]:
#Chuyển dữ liệu từ text sang số
from pyspark.ml.feature import StringIndexer
Platform_indexer = StringIndexer(inputCol='Churn', outputCol = 'Churn_Num').fit(data)
data = Platform_indexer.transform(data)

In [16]:
#Chuyển dữ liệu từ text sang số
from pyspark.ml.feature import StringIndexer
Platform_indexer = StringIndexer(inputCol='Contract', outputCol = 'Contract_Num').fit(data)
data = Platform_indexer.transform(data)

## Vector hóa đặc trưng (VectorAssembler), nghĩa là chuyển dữ liệu thành các véc tơ đặc trưng
### Chú ý: Lựa chọn các trường đặc trưng đã được số hóa: 

In [17]:
from pyspark.ml.feature import VectorAssembler
data_assembler = VectorAssembler(inputCols=['InternetService_Num', 'PaymentMethod_Num', 'gender_Num', 'Churn_Num', 'Contract_Num', 'MonthlyCharges', 'tenure' ], outputCol='features')

#gán lại dữ liệu đã được biến đổi
data = data_assembler.transform(data)

## Khai báo bộ học máy phân lớp

In [18]:
from pyspark.ml.classification import LogisticRegression

## chia dữ liệu thành hai tập: huấn luyện và kiểm tra
### lựa chọn dữ liệu (đặc trưng và mục tiêu/nhãn)

In [19]:
data_model = data.select(['features', 'Churn_Num']).show(10, False)

+--------------------------------+---------+
|features                        |Churn_Num|
+--------------------------------+---------+
|[1.0,0.0,1.0,0.0,0.0,29.85,1.0] |0.0      |
|[1.0,1.0,0.0,0.0,2.0,56.95,34.0]|0.0      |
|[1.0,1.0,0.0,1.0,0.0,53.85,2.0] |1.0      |
|[1.0,2.0,0.0,0.0,2.0,42.3,45.0] |0.0      |
|[0.0,0.0,1.0,1.0,0.0,70.7,2.0]  |1.0      |
|[0.0,0.0,1.0,1.0,0.0,99.65,8.0] |1.0      |
|(7,[1,5,6],[3.0,89.1,22.0])     |0.0      |
|[1.0,1.0,1.0,0.0,0.0,29.75,10.0]|0.0      |
|[0.0,0.0,1.0,1.0,0.0,104.8,28.0]|1.0      |
|[1.0,2.0,0.0,0.0,2.0,56.15,62.0]|0.0      |
+--------------------------------+---------+
only showing top 10 rows



## chia dữ liệu cho mô hình học máy

In [20]:
# chia dữ liệu học máy
model_data = data.select(['features','Churn_Num'])
train_data,test_data = model_data.randomSplit([0.7,0.3])
print(train_data.count())
train_data.groupBy('Churn_Num').count().show()
# tỉ lệ chia dữ liệu: tập huấn luyện chiếm 70% và tập kiểm tra chiếm 30%
# xem dữ liệu
train_data.count(), test_data.count()

4974
+---------+-----+
|Churn_Num|count|
+---------+-----+
|      0.0| 3658|
|      1.0| 1316|
+---------+-----+



(4974, 2069)

## Huấn luyện mô hình

In [21]:
model = LogisticRegression(labelCol = 'Churn_Num').fit(train_data)

## Đánh giá mô hình học máy

In [22]:
result = model.evaluate(test_data).predictions

#xem kết quả 10 dòng đầu tiên của dữ liệu kiểm tra
result.select(['Churn_Num', 'prediction']).show(10, False)

C:\spark\spark\spark-3.2.3-bin-hadoop2.7\python\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+---------+----------+
|Churn_Num|prediction|
+---------+----------+
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
|0.0      |0.0       |
+---------+----------+
only showing top 10 rows



## Chỉ số đánh giá mô hình (Precision và Recall)

In [23]:
#true postives - thực đúng
TP = result[(result.Churn_Num == 1) & (result.prediction == 1)].count()
#true negative - thực sai
TN = result[(result.Churn_Num == 0) & (result.prediction == 0)].count()
#false postives - sai nhưng dự báo đúng
FP = result[(result.Churn_Num == 0) & (result.prediction == 1)].count()
#false negative - sai và dự báo là sai
FN = result[(result.Churn_Num == 1) & (result.prediction == 0)].count()

recall = TP/(TP+FN)
precision = TP/(TP+FP)
accuracy = (TP+TN)/(TP+TN+FP+FN)

print(recall)
print(precision)
print(accuracy)

1.0
1.0
1.0
